# Analyse de Drift en Production

Ce notebook analyse les logs de production de l'API de scoring crédit pour détecter :
- Le **data drift** (dérive des features d'entrée) via Evidently AI
- Le **prediction drift** (dérive des sorties du modèle)
- Les **métriques opérationnelles** (latence, taux d'erreur)

In [ ]:
import json
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from evidently.metric_preset import DataDriftPreset
from evidently.report import Report

sns.set_theme(style="whitegrid")
pd.set_option("display.max_columns", 15)

In [ ]:
# Load reference data (training distribution)
FEATURE_COLUMNS = [
    "EXT_SOURCES_MEAN", "CREDIT_TERM", "EXT_SOURCE_3",
    "GOODS_PRICE_CREDIT_PERCENT", "INSTAL_AMT_PAYMENT_sum",
    "AMT_ANNUITY", "POS_CNT_INSTALMENT_FUTURE_mean",
    "DAYS_BIRTH", "EXT_SOURCES_WEIGHTED", "EXT_SOURCE_2",
]

ref_data = pd.read_csv("../data/dataset_top10_features_data.csv")
ref_features = ref_data[FEATURE_COLUMNS]

print(f"Reference data: {ref_features.shape[0]:,} rows, {ref_features.shape[1]} features")
ref_features.describe()

In [ ]:
# Load production logs
log_path = Path("../logs/predictions.jsonl")

logs = []
with open(log_path) as f:
    for line in f:
        logs.append(json.loads(line))

prod_df = pd.DataFrame(logs)
prod_df["timestamp"] = pd.to_datetime(prod_df["timestamp"])

print(f"Total log entries: {len(prod_df)}")
print(f"Period: {prod_df['timestamp'].min()} to {prod_df['timestamp'].max()}")
print(f"Status codes: {prod_df['status_code'].value_counts().to_dict()}")

# Filter successful predictions and extract features
success_df = prod_df[prod_df["status_code"] == 200].copy()
prod_features = pd.DataFrame(success_df["input_features"].tolist())
prod_features = prod_features[FEATURE_COLUMNS]

print(f"\nSuccessful predictions: {len(prod_features):,}")
prod_features.describe()

In [ ]:
# Evidently Data Drift Report
drift_report = Report(metrics=[DataDriftPreset()])
drift_report.run(reference_data=ref_features, current_data=prod_features)
drift_report

In [ ]:
# Save drift report as HTML
report_path = Path("data_drift_report.html")
drift_report.save_html(str(report_path))
print(f"Drift report saved to {report_path.resolve()}")

In [ ]:
# Distribution comparison: Reference vs Production
fig, axes = plt.subplots(5, 2, figsize=(16, 20))
axes = axes.flatten()

for i, col in enumerate(FEATURE_COLUMNS):
    ax = axes[i]
    ax.hist(ref_features[col].dropna(), bins=50, alpha=0.5, label="Référence", density=True, color="steelblue")
    ax.hist(prod_features[col].dropna(), bins=50, alpha=0.5, label="Production", density=True, color="coral")
    ax.set_title(col, fontsize=11, fontweight="bold")
    ax.legend(fontsize=9)
    ax.set_ylabel("Densité")

fig.suptitle("Distribution des features : Référence vs Production", fontsize=14, fontweight="bold", y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# Prediction drift analysis
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Probability distribution
axes[0].hist(success_df["probability_default"], bins=50, color="steelblue", edgecolor="white")
axes[0].axvline(x=0.10, color="red", linestyle="--", label="Seuil (0.10)")
axes[0].set_title("Distribution des probabilités de défaut", fontweight="bold")
axes[0].set_xlabel("Probabilité de défaut")
axes[0].set_ylabel("Nombre de prédictions")
axes[0].legend()

# Decision counts
decision_counts = success_df["credit_decision"].value_counts()
colors = ["#2ecc71" if d == "approved" else "#e74c3c" for d in decision_counts.index]
axes[1].bar(decision_counts.index, decision_counts.values, color=colors)
axes[1].set_title("Décisions de crédit", fontweight="bold")
axes[1].set_ylabel("Nombre")
for j, v in enumerate(decision_counts.values):
    axes[1].text(j, v + 5, str(v), ha="center", fontweight="bold")

# Rolling average of probability over time
ts_sorted = success_df.sort_values("timestamp").reset_index(drop=True)
ts_sorted["prob_rolling"] = ts_sorted["probability_default"].rolling(window=50, min_periods=1).mean()
axes[2].plot(ts_sorted["timestamp"], ts_sorted["prob_rolling"], color="steelblue", linewidth=1.5)
axes[2].axhline(y=0.10, color="red", linestyle="--", label="Seuil (0.10)")
axes[2].set_title("Probabilité moyenne glissante (fenêtre=50)", fontweight="bold")
axes[2].set_xlabel("Temps")
axes[2].set_ylabel("Probabilité moyenne")
axes[2].legend()
axes[2].tick_params(axis="x", rotation=30)

plt.tight_layout()
plt.show()

In [ ]:
# Operational monitoring
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Error rate by hour
prod_df["hour"] = prod_df["timestamp"].dt.floor("h")
hourly = prod_df.groupby("hour").agg(
    total=("status_code", "count"),
    errors=("status_code", lambda x: (x >= 400).sum()),
)
hourly["error_rate"] = hourly["errors"] / hourly["total"]
axes[0].bar(range(len(hourly)), hourly["error_rate"], color="coral", alpha=0.8)
axes[0].set_title("Taux d'erreur par heure", fontweight="bold")
axes[0].set_xlabel("Heure (index)")
axes[0].set_ylabel("Taux d'erreur")
axes[0].axhline(y=0.05, color="red", linestyle="--", alpha=0.5, label="Seuil 5%")
axes[0].legend()

# Latency distribution
p95 = prod_df["duration_ms"].quantile(0.95)
axes[1].hist(prod_df["duration_ms"], bins=50, color="steelblue", edgecolor="white")
axes[1].axvline(x=p95, color="red", linestyle="--", label=f"P95 = {p95:.0f}ms")
axes[1].set_title("Distribution de la latence", fontweight="bold")
axes[1].set_xlabel("Latence (ms)")
axes[1].set_ylabel("Nombre de requêtes")
axes[1].legend()

# Latency over time
ts_all = prod_df.sort_values("timestamp").reset_index(drop=True)
ts_all["latency_rolling"] = ts_all["duration_ms"].rolling(window=50, min_periods=1).mean()
axes[2].plot(ts_all["timestamp"], ts_all["latency_rolling"], color="steelblue", linewidth=1.5)
axes[2].axhline(y=p95, color="red", linestyle="--", alpha=0.5, label=f"P95 = {p95:.0f}ms")
axes[2].set_title("Latence moyenne glissante (fenêtre=50)", fontweight="bold")
axes[2].set_xlabel("Temps")
axes[2].set_ylabel("Latence (ms)")
axes[2].legend()
axes[2].tick_params(axis="x", rotation=30)

plt.tight_layout()
plt.show()

In [ ]:
# Summary statistics
total = len(prod_df)
errors = (prod_df["status_code"] >= 400).sum()
ok = total - errors
denied = (success_df["credit_decision"] == "denied").sum()

print("=" * 50)
print("  TABLEAU DE BORD - MÉTRIQUES OPÉRATIONNELLES")
print("=" * 50)
print(f"  Requêtes totales :       {total:,}")
print(f"  Succès :                 {ok:,}")
print(f"  Erreurs :                {errors:,} ({errors/total:.1%})")
print(f"  Crédits refusés :        {denied:,}/{ok:,} ({denied/ok:.1%})")
print()
print("  Latence (ms) :")
print(f"    Médiane (P50) :        {prod_df['duration_ms'].quantile(0.50):.1f}")
print(f"    P90 :                  {prod_df['duration_ms'].quantile(0.90):.1f}")
print(f"    P95 :                  {prod_df['duration_ms'].quantile(0.95):.1f}")
print(f"    P99 :                  {prod_df['duration_ms'].quantile(0.99):.1f}")
print("=" * 50)

## Recommandations

### Points de vigilance - Drift détecté

L'analyse Evidently révèle un drift significatif sur **3 features** :

| Feature | Type de drift | Cause probable | Impact |
|---------|--------------|----------------|--------|
| `EXT_SOURCE_2` | Shift vers le bas (-0.15) | Changement de scoring du bureau de crédit | Augmentation des scores de risque |
| `DAYS_BIRTH` | Shift vers 0 (+3000 jours) | Rajeunissement de la clientèle | Profil de risque différent |
| `AMT_ANNUITY` | Augmentation (+20%) | Inflation / hausse des montants | Surestimation possible du risque |

### Seuils de déclenchement de ré-entraînement

1. **Drift statistique** : Si le test de Kolmogorov-Smirnov détecte un drift sur plus de 3 features simultanément (p-value < 0.05), planifier un ré-entraînement.
2. **Taux de refus** : Si le taux de refus dépasse 50% sur une fenêtre glissante de 7 jours, investiguer.
3. **Performance opérationnelle** : Si le taux d'erreur dépasse 5% ou la latence P95 dépasse 500ms, alerter.

### Actions recommandées

- **Court terme** : Recalibrer le seuil de décision (actuellement 0.10) sur les données récentes
- **Moyen terme** : Ré-entraîner le modèle en incluant les données de production récentes
- **Long terme** : Automatiser la détection de drift avec des alertes (Evidently + cron job ou Airflow)